[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/danpele/Time-Series-Analysis/blob/main/chapter5_lecture_notebook.ipynb)

---

# Chapter 5: VAR Models and Granger Causality

**Course:** Time Series Analysis and Forecasting  
**Program:** Bachelor program, Faculty of Cybernetics, Statistics and Economic Informatics, Bucharest University of Economic Studies, Romania  
**Academic Year:** 2025-2026

---

## Learning Objectives

By the end of this notebook, you will be able to:
1. Understand the structure of Vector Autoregression (VAR) models
2. Estimate VAR models and select optimal lag order
3. Conduct and interpret Granger causality tests
4. Compute and interpret Impulse Response Functions (IRF)
5. Perform Forecast Error Variance Decomposition (FEVD)
6. Understand cointegration and Vector Error Correction Models (VECM)

## Setup and Imports

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# VAR and multivariate time series
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller, grangercausalitytests, ccf
from statsmodels.tsa.vector_ar.vecm import coint_johansen, VECM
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy import stats

# For fetching real data
try:
    import pandas_datareader.data as web
    HAS_PDR = True
except ImportError:
    HAS_PDR = False
    print("Note: pandas_datareader not installed. Install with: pip install pandas-datareader")

# Plotting style - clean, professional
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['savefig.facecolor'] = 'none'
plt.rcParams['axes.grid'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# Colors (IDA color scheme)
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666'
}

print("All libraries loaded successfully!")

## 1. Introduction to Multivariate Time Series

In many applications, we have **multiple time series** that are related:
- GDP, consumption, investment, government spending
- Stock prices of related companies
- Interest rates at different maturities
- Inflation and unemployment (Phillips curve)

**Why multivariate models?**
- Capture interdependencies between variables
- Improve forecasts by using information from related series
- Analyze dynamic relationships (causality, impulse responses)

In [ ]:
# Fetch REAL macroeconomic data from FRED
# GDP Growth (quarterly, annualized) and CPI Inflation (monthly → quarterly)

start_date = '1990-01-01'
end_date = '2024-12-31'

if HAS_PDR:
    try:
        # Fetch from FRED (Federal Reserve Economic Data)
        # A191RL1Q225SBEA = Real GDP Growth Rate (quarterly, annualized)
        # CPIAUCSL = Consumer Price Index (monthly)
        
        gdp_growth = web.DataReader('A191RL1Q225SBEA', 'fred', start_date, end_date)
        cpi = web.DataReader('CPIAUCSL', 'fred', start_date, end_date)
        
        # Convert CPI to quarterly and compute year-over-year inflation
        cpi_quarterly = cpi.resample('QE').last()
        inflation = cpi_quarterly.pct_change(4) * 100  # Year-over-year % change
        
        # Align and combine
        data = pd.DataFrame({
            'GDP_Growth': gdp_growth['A191RL1Q225SBEA'],
            'Inflation': inflation['CPIAUCSL']
        }).dropna()
        
        print(f"✓ Real FRED Data Loaded: {len(data)} quarterly observations")
        print(f"  Period: {data.index[0].strftime('%Y-Q%q')} to {data.index[-1].strftime('%Y-Q%q')}")
        DATA_SOURCE = "FRED (Federal Reserve Economic Data)"
        
    except Exception as e:
        print(f"Could not fetch FRED data: {e}")
        print("Using simulated data as fallback...")
        HAS_PDR = False

if not HAS_PDR:
    # Fallback: Simulated data
    np.random.seed(42)
    n = 140  # ~35 years quarterly
    
    Y1 = np.zeros(n)  # GDP growth
    Y2 = np.zeros(n)  # Inflation
    
    for t in range(1, n):
        Y1[t] = 0.5 * Y1[t-1] + 0.15 * Y2[t-1] + np.random.randn() * 1.5 + 2.5
        Y2[t] = 0.1 * Y1[t-1] + 0.7 * Y2[t-1] + np.random.randn() * 0.8 + 1.5
    
    data = pd.DataFrame({
        'GDP_Growth': Y1,
        'Inflation': Y2
    }, index=pd.date_range('1990-01', periods=n, freq='QE'))
    
    DATA_SOURCE = "Simulated (fallback)"
    print(f"Using simulated data: {len(data)} observations")

print(f"\nData Source: {DATA_SOURCE}")
print(data.describe().round(2))

In [ ]:
# Plot the multivariate time series
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].plot(data.index, data['GDP_Growth'], color=COLORS['blue'], linewidth=1, label='GDP Growth')
axes[0].axhline(y=data['GDP_Growth'].mean(), color='red', linestyle='--', alpha=0.5, label=f'Mean: {data["GDP_Growth"].mean():.1f}%')
axes[0].axhline(y=0, color='black', linestyle='-', alpha=0.2)
axes[0].set_title(f'US Real GDP Growth Rate (Quarterly, Annualized) - {DATA_SOURCE}', fontweight='bold')
axes[0].set_ylabel('% Change')
axes[0].legend(loc='upper right', frameon=False)

axes[1].plot(data.index, data['Inflation'], color=COLORS['orange'], linewidth=1, label='Inflation')
axes[1].axhline(y=data['Inflation'].mean(), color='red', linestyle='--', alpha=0.5, label=f'Mean: {data["Inflation"].mean():.1f}%')
axes[1].set_title('US CPI Inflation Rate (Year-over-Year)', fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('% Change')
axes[1].legend(loc='upper right', frameon=False)

plt.tight_layout()
plt.show()

# Correlation and basic statistics
print(f"\nCorrelation between GDP Growth and Inflation: {data['GDP_Growth'].corr(data['Inflation']):.4f}")
print(f"\nNote: Using REAL macroeconomic data allows us to discover actual")
print(f"      economic relationships rather than recovering known parameters.")

## 2. The VAR(p) Model

A **Vector Autoregression of order p**, VAR(p), for $K$ variables is:

$$\mathbf{Y}_t = \mathbf{c} + \mathbf{A}_1 \mathbf{Y}_{t-1} + \mathbf{A}_2 \mathbf{Y}_{t-2} + \cdots + \mathbf{A}_p \mathbf{Y}_{t-p} + \boldsymbol{\varepsilon}_t$$

where:
- $\mathbf{Y}_t$ is a $K \times 1$ vector of endogenous variables
- $\mathbf{c}$ is a $K \times 1$ vector of constants
- $\mathbf{A}_i$ are $K \times K$ coefficient matrices
- $\boldsymbol{\varepsilon}_t$ is a $K \times 1$ vector of white noise errors

### Number of Parameters
- Each equation has: $1 + Kp$ parameters (constant + K coefficients × p lags)
- Total system: $K(1 + Kp)$ parameters
- Plus $K(K+1)/2$ covariance parameters

In [ ]:
# VAR(1) in matrix form
print("VAR(1) for 2 Variables:")
print("="*60)
print()
print("[ Y1_t ]   [ c1 ]   [ a11  a12 ] [ Y1_{t-1} ]   [ e1_t ]")
print("[      ] = [    ] + [          ] [          ] + [      ]")
print("[ Y2_t ]   [ c2 ]   [ a21  a22 ] [ Y2_{t-1} ]   [ e2_t ]")
print()
print("Written as two equations:")
print("  Y1_t = c1 + a11*Y1_{t-1} + a12*Y2_{t-1} + e1_t")
print("  Y2_t = c2 + a21*Y1_{t-1} + a22*Y2_{t-1} + e2_t")
print()
print(f"Parameters per equation: 1 + K*p = 1 + 2*1 = 3")
print(f"Total parameters: K*(1 + K*p) = 2*(1 + 2*1) = 6")

## 3. VAR Stability Condition

A VAR(p) is **stable** (stationary) if all eigenvalues of the companion matrix lie inside the unit circle:

$$|\lambda_i| < 1 \quad \text{for all } i$$

### Companion Form
Any VAR(p) can be written as a VAR(1) in companion form:

$$\mathbf{Z}_t = \mathbf{A} \mathbf{Z}_{t-1} + \mathbf{u}_t$$

where $\mathbf{Z}_t = [\mathbf{Y}_t', \mathbf{Y}_{t-1}', ..., \mathbf{Y}_{t-p+1}']'$

In [ ]:
# Demonstrate stability condition with an example
# We'll check stability after fitting the model to real data

# For illustration, show a stable vs unstable example
print("Example: Stable vs Unstable VAR(1) Systems")
print("="*60)

# Stable system
A_stable = np.array([[0.6, 0.2],
                     [0.1, 0.5]])
eig_stable = np.linalg.eigvals(A_stable)

# Unstable system (unit root)
A_unstable = np.array([[0.9, 0.2],
                       [0.1, 0.95]])
eig_unstable = np.linalg.eigvals(A_unstable)

print("\nStable System:")
print(f"  A = [[0.6, 0.2], [0.1, 0.5]]")
print(f"  Eigenvalues: {eig_stable.round(3)}")
print(f"  Moduli: {np.abs(eig_stable).round(3)}")
print(f"  Stable: {all(np.abs(eig_stable) < 1)} ✓")

print("\nUnstable System:")
print(f"  A = [[0.9, 0.2], [0.1, 0.95]]")
print(f"  Eigenvalues: {eig_unstable.round(3)}")
print(f"  Moduli: {np.abs(eig_unstable).round(3)}")
print(f"  Stable: {all(np.abs(eig_unstable) < 1)} ✗")

# Visualize in complex plane
fig, ax = plt.subplots(figsize=(7, 7))

# Unit circle
theta = np.linspace(0, 2*np.pi, 100)
ax.plot(np.cos(theta), np.sin(theta), 'k--', alpha=0.5, label='Unit Circle')

# Eigenvalues
ax.scatter(eig_stable.real, eig_stable.imag, s=200, c=COLORS['green'], 
           marker='o', linewidths=2, label='Stable (inside)', zorder=5)
ax.scatter(eig_unstable.real, eig_unstable.imag, s=200, c=COLORS['red'], 
           marker='x', linewidths=3, label='Unstable (on/outside)', zorder=5)

ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_aspect('equal')
ax.axhline(y=0, color='gray', linewidth=0.5)
ax.axvline(x=0, color='gray', linewidth=0.5)
ax.set_xlabel('Real')
ax.set_ylabel('Imaginary')
ax.set_title('VAR Stability: Eigenvalues and the Unit Circle', fontweight='bold')
ax.legend(loc='upper left', frameon=False)

plt.tight_layout()
plt.show()

## 4. Estimating VAR Models

In [ ]:
# Fit VAR model
model = VAR(data)

# Select optimal lag order
lag_selection = model.select_order(maxlags=8)
print("Lag Order Selection Criteria:")
print(lag_selection.summary())

In [ ]:
# Fit VAR(1) based on BIC
results = model.fit(1)
print(results.summary())

In [ ]:
# Examine estimated coefficients and check stability
print("Estimated VAR Coefficients:")
print("="*50)

# Get coefficient matrices
for i, coef in enumerate(results.coefs):
    print(f"\nLag {i+1} coefficient matrix A_{i+1}:")
    print(pd.DataFrame(coef, 
                       index=data.columns,
                       columns=data.columns).round(4))

# Check stability of estimated model
print("\n" + "="*50)
print("Stability Check:")
print("="*50)

# For VAR(p), stability is checked via companion matrix
# statsmodels provides this
roots = results.roots
print(f"\nCharacteristic roots (inverse eigenvalues):")
for i, root in enumerate(roots):
    print(f"  Root {i+1}: {root:.4f} (modulus: {np.abs(root):.4f})")

print(f"\nVAR is stable: {all(np.abs(roots) < 1)}")

if all(np.abs(roots) < 1):
    print("✓ All roots inside unit circle - model is stable")
else:
    print("✗ Warning: Some roots on or outside unit circle")

## 5. Granger Causality

**Granger causality** tests whether lagged values of one variable help predict another.

### Definition
$X$ **Granger-causes** $Y$ if:
- Past values of $X$ contain information useful for predicting $Y$
- Beyond what is already contained in past values of $Y$ itself

### The Test
In a VAR with $Y$ and $X$:
$$Y_t = c + \sum_{i=1}^p \alpha_i Y_{t-i} + \sum_{i=1}^p \beta_i X_{t-i} + \varepsilon_t$$

Test $H_0: \beta_1 = \beta_2 = \cdots = \beta_p = 0$ (X does NOT Granger-cause Y)

### Important Caveats
- Granger causality ≠ true causality
- May be spurious due to omitted variables
- Sensitive to lag selection

In [ ]:
# Granger causality tests
print("Granger Causality Tests")
print("="*60)

# Test: Does Inflation Granger-cause GDP Growth?
print("\n1. H₀: Inflation does NOT Granger-cause GDP Growth")
print("-"*50)
gc_infl_to_gdp = grangercausalitytests(data[['GDP_Growth', 'Inflation']], maxlag=4, verbose=True)

In [ ]:
# Test: Does GDP Growth Granger-cause Inflation?
print("\n2. H₀: GDP Growth does NOT Granger-cause Inflation")
print("-"*50)
gc_gdp_to_infl = grangercausalitytests(data[['Inflation', 'GDP_Growth']], maxlag=4, verbose=True)

In [ ]:
# Summary of Granger causality
print("\nGranger Causality Summary (at lag 1):")
print("="*50)

p_infl_to_gdp = gc_infl_to_gdp[1][0]['ssr_ftest'][1]
p_gdp_to_infl = gc_gdp_to_infl[1][0]['ssr_ftest'][1]

print(f"Inflation → GDP: p-value = {p_infl_to_gdp:.4f} {'✓ Significant' if p_infl_to_gdp < 0.05 else '✗ Not significant'}")
print(f"GDP → Inflation: p-value = {p_gdp_to_infl:.4f} {'✓ Significant' if p_gdp_to_infl < 0.05 else '✗ Not significant'}")

print("\nInterpretation:")
if p_infl_to_gdp < 0.05 and p_gdp_to_infl < 0.05:
    print("  Bidirectional causality (feedback)")
elif p_infl_to_gdp < 0.05:
    print("  Inflation Granger-causes GDP (unidirectional)")
elif p_gdp_to_infl < 0.05:
    print("  GDP Granger-causes Inflation (unidirectional)")
else:
    print("  No Granger causality detected")

## 6. Impulse Response Functions (IRF)

**Impulse Response Functions** trace the effect of a one-time shock to one variable on all variables over time.

$$\text{IRF}_{ij}(h) = \frac{\partial Y_{i,t+h}}{\partial \varepsilon_{j,t}}$$

### Key Properties
- Shows dynamic multipliers
- For stable VAR: IRF → 0 as h → ∞
- Requires identification (e.g., Cholesky ordering)

In [ ]:
# Compute IRFs
irf = results.irf(20)

# Plot IRFs
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Response to GDP shock
axes[0, 0].plot(irf.irfs[:, 0, 0], color=COLORS['blue'], linewidth=2, label='IRF')
axes[0, 0].fill_between(range(21), 
                        irf.irfs[:, 0, 0] - 1.96*irf.stderr()[:, 0, 0],
                        irf.irfs[:, 0, 0] + 1.96*irf.stderr()[:, 0, 0], 
                        alpha=0.2, color=COLORS['blue'], label='95% CI')
axes[0, 0].axhline(y=0, color='black', linestyle='-', alpha=0.3)
axes[0, 0].set_title('GDP → GDP', fontweight='bold')
axes[0, 0].legend(loc='upper right', frameon=False)

axes[0, 1].plot(irf.irfs[:, 1, 0], color=COLORS['blue'], linewidth=2, label='IRF')
axes[0, 1].fill_between(range(21), 
                        irf.irfs[:, 1, 0] - 1.96*irf.stderr()[:, 1, 0],
                        irf.irfs[:, 1, 0] + 1.96*irf.stderr()[:, 1, 0], 
                        alpha=0.2, color=COLORS['blue'], label='95% CI')
axes[0, 1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
axes[0, 1].set_title('GDP → Inflation', fontweight='bold')
axes[0, 1].legend(loc='upper right', frameon=False)

# Response to Inflation shock
axes[1, 0].plot(irf.irfs[:, 0, 1], color=COLORS['orange'], linewidth=2, label='IRF')
axes[1, 0].fill_between(range(21), 
                        irf.irfs[:, 0, 1] - 1.96*irf.stderr()[:, 0, 1],
                        irf.irfs[:, 0, 1] + 1.96*irf.stderr()[:, 0, 1], 
                        alpha=0.2, color=COLORS['orange'], label='95% CI')
axes[1, 0].axhline(y=0, color='black', linestyle='-', alpha=0.3)
axes[1, 0].set_title('Inflation → GDP', fontweight='bold')
axes[1, 0].set_xlabel('Horizon (months)')
axes[1, 0].legend(loc='upper right', frameon=False)

axes[1, 1].plot(irf.irfs[:, 1, 1], color=COLORS['orange'], linewidth=2, label='IRF')
axes[1, 1].fill_between(range(21), 
                        irf.irfs[:, 1, 1] - 1.96*irf.stderr()[:, 1, 1],
                        irf.irfs[:, 1, 1] + 1.96*irf.stderr()[:, 1, 1], 
                        alpha=0.2, color=COLORS['orange'], label='95% CI')
axes[1, 1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
axes[1, 1].set_title('Inflation → Inflation', fontweight='bold')
axes[1, 1].set_xlabel('Horizon (months)')
axes[1, 1].legend(loc='upper right', frameon=False)

plt.tight_layout()
plt.show()

print("IRF Interpretation:")
print("- Own shocks have immediate impact, then decay")
print("- Cross shocks show spillover effects")
print("- All responses converge to 0 (stable VAR)")

## 7. Forecast Error Variance Decomposition (FEVD)

**FEVD** decomposes the variance of forecast errors into contributions from each shock.

$$\text{FEVD}_{ij}(h) = \frac{\text{Variance of } Y_i \text{ due to shock } j}{\text{Total variance of } Y_i}$$

### Interpretation
- Shows relative importance of each shock
- At h=0: own shock explains 100%
- As h increases: shows long-run importance

In [ ]:
# Compute FEVD
fevd = results.fevd(20)

# Get the number of periods and variables
n_periods = fevd.periods + 1  # +1 because it includes period 0
n_vars = len(data.columns)
horizons = range(n_periods)

# Extract FEVD data properly - fevd.decomp shape is (periods, neqs, neqs)
# fevd.decomp[h, i, j] = fraction of variance of variable i explained by shock j at horizon h

# Plot FEVD
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# FEVD for GDP Growth (variable 0)
gdp_by_gdp = [fevd.decomp[h, 0, 0] * 100 for h in range(n_periods)]
gdp_by_infl = [fevd.decomp[h, 0, 1] * 100 for h in range(n_periods)]

axes[0].stackplot(horizons, gdp_by_gdp, gdp_by_infl,
                  labels=['GDP shock', 'Inflation shock'],
                  colors=[COLORS['blue'], COLORS['orange']], alpha=0.7)
axes[0].set_title('FEVD of GDP Growth', fontweight='bold')
axes[0].set_xlabel('Horizon (quarters)')
axes[0].set_ylabel('Percent')
axes[0].legend(loc='center right', frameon=False)
axes[0].set_ylim(0, 100)

# FEVD for Inflation (variable 1)
infl_by_gdp = [fevd.decomp[h, 1, 0] * 100 for h in range(n_periods)]
infl_by_infl = [fevd.decomp[h, 1, 1] * 100 for h in range(n_periods)]

axes[1].stackplot(horizons, infl_by_gdp, infl_by_infl,
                  labels=['GDP shock', 'Inflation shock'],
                  colors=[COLORS['blue'], COLORS['orange']], alpha=0.7)
axes[1].set_title('FEVD of Inflation', fontweight='bold')
axes[1].set_xlabel('Horizon (quarters)')
axes[1].set_ylabel('Percent')
axes[1].legend(loc='center right', frameon=False)
axes[1].set_ylim(0, 100)

plt.tight_layout()
plt.show()

# Print table
print("\nFEVD Table (%)")
print("="*70)
print(f"{'Horizon':<10} {'GDP by GDP':>12} {'GDP by Infl':>12} {'Infl by GDP':>12} {'Infl by Infl':>12}")
print("-"*70)
for h in [1, 5, 10, min(20, n_periods-1)]:
    if h < n_periods:
        print(f"{h:<10} {fevd.decomp[h, 0, 0]*100:>12.1f} {fevd.decomp[h, 0, 1]*100:>12.1f} "
              f"{fevd.decomp[h, 1, 0]*100:>12.1f} {fevd.decomp[h, 1, 1]*100:>12.1f}")

## 8. VAR Forecasting

In [ ]:
# Generate forecasts
forecast_steps = 12
lag_order = results.k_ar

forecast = results.forecast(data.values[-lag_order:], steps=forecast_steps)
forecast_interval = results.forecast_interval(data.values[-lag_order:], steps=forecast_steps, alpha=0.05)

# Create forecast dates
forecast_dates = pd.date_range(start=data.index[-1] + pd.DateOffset(months=1), 
                               periods=forecast_steps, freq='ME')

# Plot
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# GDP Growth
axes[0].plot(data.index[-36:], data['GDP_Growth'].values[-36:], 
             color=COLORS['blue'], linewidth=1.5, label='Historical')
axes[0].plot(forecast_dates, forecast[:, 0], 
             color=COLORS['red'], linewidth=2, linestyle='--', label='Forecast')
axes[0].fill_between(forecast_dates, forecast_interval[1][:, 0], forecast_interval[2][:, 0],
                     color=COLORS['red'], alpha=0.2, label='95% CI')
axes[0].axvline(x=data.index[-1], color='black', linestyle='-', alpha=0.3)
axes[0].set_title('GDP Growth Forecast', fontweight='bold')
axes[0].set_ylabel('%')
axes[0].legend(loc='upper right', frameon=False)

# Inflation
axes[1].plot(data.index[-36:], data['Inflation'].values[-36:], 
             color=COLORS['orange'], linewidth=1.5, label='Historical')
axes[1].plot(forecast_dates, forecast[:, 1], 
             color=COLORS['red'], linewidth=2, linestyle='--', label='Forecast')
axes[1].fill_between(forecast_dates, forecast_interval[1][:, 1], forecast_interval[2][:, 1],
                     color=COLORS['red'], alpha=0.2, label='95% CI')
axes[1].axvline(x=data.index[-1], color='black', linestyle='-', alpha=0.3)
axes[1].set_title('Inflation Forecast', fontweight='bold')
axes[1].set_ylabel('%')
axes[1].set_xlabel('Date')
axes[1].legend(loc='upper right', frameon=False)

plt.tight_layout()
plt.show()

# Forecast table
print("\nForecast Summary:")
print("="*70)
print(f"{'Date':<12} {'GDP':>10} {'GDP 95% CI':>20} {'Inflation':>10} {'Infl 95% CI':>20}")
print("-"*70)
for i in range(min(6, forecast_steps)):
    print(f"{str(forecast_dates[i].date()):<12} {forecast[i, 0]:>10.2f} "
          f"[{forecast_interval[1][i, 0]:>6.2f}, {forecast_interval[2][i, 0]:>6.2f}] "
          f"{forecast[i, 1]:>10.2f} [{forecast_interval[1][i, 1]:>6.2f}, {forecast_interval[2][i, 1]:>6.2f}]")

## 9. Model Diagnostics

In [ ]:
# Residual diagnostics
residuals = results.resid

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Residuals over time
axes[0, 0].plot(residuals[:, 0], color=COLORS['blue'], alpha=0.7, linewidth=0.5, label='GDP residuals')
axes[0, 0].plot(residuals[:, 1], color=COLORS['orange'], alpha=0.7, linewidth=0.5, label='Inflation residuals')
axes[0, 0].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[0, 0].set_title('Residuals Over Time', fontweight='bold')
axes[0, 0].legend(loc='upper right', frameon=False)

# Cross-correlation of residuals
axes[0, 1].scatter(residuals[:, 0], residuals[:, 1], alpha=0.5, color=COLORS['blue'], s=20)
axes[0, 1].set_xlabel('GDP Residuals')
axes[0, 1].set_ylabel('Inflation Residuals')
axes[0, 1].set_title('Residual Cross-Plot', fontweight='bold')
corr = np.corrcoef(residuals[:, 0], residuals[:, 1])[0, 1]
axes[0, 1].text(0.05, 0.95, f'Corr = {corr:.3f}', transform=axes[0, 1].transAxes, 
                fontsize=12, verticalalignment='top')

# ACF of residuals
plot_acf(residuals[:, 0], ax=axes[1, 0], lags=20, color=COLORS['blue'], title='ACF: GDP Residuals')
axes[1, 0].set_title('ACF: GDP Residuals', fontweight='bold')

plot_acf(residuals[:, 1], ax=axes[1, 1], lags=20, color=COLORS['orange'], title='ACF: Inflation Residuals')
axes[1, 1].set_title('ACF: Inflation Residuals', fontweight='bold')

plt.tight_layout()
plt.show()

# Portmanteau test
print("\nPortmanteau Test for Residual Autocorrelation:")
print(results.test_whiteness(nlags=12).summary())

## 10. Cointegration and VECM

When variables are **I(1)** (non-stationary) but share a long-run equilibrium, they are **cointegrated**.

### Vector Error Correction Model (VECM)
$$\Delta \mathbf{Y}_t = \boldsymbol{\Pi} \mathbf{Y}_{t-1} + \sum_{i=1}^{p-1} \boldsymbol{\Gamma}_i \Delta \mathbf{Y}_{t-i} + \boldsymbol{\varepsilon}_t$$

where $\boldsymbol{\Pi} = \boldsymbol{\alpha} \boldsymbol{\beta}'$ contains:
- $\boldsymbol{\beta}$: cointegrating vectors (long-run relationships)
- $\boldsymbol{\alpha}$: adjustment speeds (error correction)

In [ ]:
# Cointegration Example: US Interest Rates
# Short-term (3-month T-bill) and Long-term (10-year Treasury) yields

if HAS_PDR:
    try:
        # Fetch Treasury rates from FRED
        # TB3MS = 3-Month Treasury Bill
        # GS10 = 10-Year Treasury Constant Maturity Rate
        
        short_rate = web.DataReader('TB3MS', 'fred', '1990-01-01', '2024-12-31')
        long_rate = web.DataReader('GS10', 'fred', '1990-01-01', '2024-12-31')
        
        # Align to monthly
        coint_data = pd.DataFrame({
            'ShortRate': short_rate['TB3MS'],
            'LongRate': long_rate['GS10']
        }).dropna()
        
        print(f"✓ Real Interest Rate Data: {len(coint_data)} monthly observations")
        COINT_SOURCE = "FRED Treasury Rates"
        
    except Exception as e:
        print(f"Could not fetch rate data: {e}")
        HAS_PDR = False

if not HAS_PDR:
    # Fallback: Simulated cointegrated data
    np.random.seed(123)
    n = 200
    
    x = np.cumsum(np.random.randn(n)) + 50
    y = 2 * x + np.random.randn(n) * 2 + 10
    
    coint_data = pd.DataFrame({'ShortRate': x, 'LongRate': y})
    COINT_SOURCE = "Simulated (fallback)"

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(coint_data.index, coint_data['ShortRate'], color=COLORS['blue'], 
             label='3-Month T-Bill', linewidth=1)
axes[0].plot(coint_data.index, coint_data['LongRate'], color=COLORS['orange'], 
             label='10-Year Treasury', linewidth=1)
axes[0].set_title(f'US Interest Rates - {COINT_SOURCE}', fontweight='bold')
axes[0].set_ylabel('Interest Rate (%)')
axes[0].legend(loc='upper right', frameon=False)

# Spread (term spread)
spread = coint_data['LongRate'] - coint_data['ShortRate']
axes[1].plot(spread.index, spread, color=COLORS['green'], linewidth=1)
axes[1].axhline(y=spread.mean(), color='red', linestyle='--', alpha=0.5, 
                label=f'Mean: {spread.mean():.2f}%')
axes[1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
axes[1].set_title('Term Spread: Long Rate - Short Rate', fontweight='bold')
axes[1].set_ylabel('Spread (%)')
axes[1].legend(loc='upper right', frameon=False)

plt.tight_layout()
plt.show()

# ADF tests
print("\nUnit Root Tests (ADF):")
print("="*60)
for col in ['ShortRate', 'LongRate']:
    result = adfuller(coint_data[col].dropna())
    status = "Stationary" if result[1] < 0.05 else "Non-stationary (I(1))"
    print(f"{col:>12}: ADF = {result[0]:>7.3f}, p-value = {result[1]:.4f} → {status}")

result = adfuller(spread.dropna())
status = "Stationary" if result[1] < 0.05 else "Non-stationary"
print(f"{'Spread':>12}: ADF = {result[0]:>7.3f}, p-value = {result[1]:.4f} → {status}")

print("\nIf rates are I(1) but spread is I(0), the rates are cointegrated!")

In [ ]:
# Johansen cointegration test
print("\nJohansen Cointegration Test:")
print("="*60)

johansen_result = coint_johansen(coint_data, det_order=0, k_ar_diff=1)

print("\nTrace Statistics:")
print(f"{'Rank':>6} {'Trace Stat':>12} {'Crit 95%':>12} {'Crit 99%':>12}")
print("-"*50)
for i in range(2):
    sig = " **" if johansen_result.lr1[i] > johansen_result.cvt[i, 1] else ""
    print(f"{i:>6} {johansen_result.lr1[i]:>12.2f} {johansen_result.cvt[i, 1]:>12.2f} "
          f"{johansen_result.cvt[i, 2]:>12.2f}{sig}")

print("\nConclusion: Reject rank=0, cannot reject rank=1")
print("→ There is 1 cointegrating relationship")

In [ ]:
# Fit VECM
vecm = VECM(coint_data, k_ar_diff=1, coint_rank=1, deterministic='ci')
vecm_results = vecm.fit()

print("\nVECM Estimation Results:")
print("="*60)
print(vecm_results.summary())

## 11. Cross-Correlation Function

In [ ]:
# Cross-correlation between GDP and Inflation
from scipy import signal

# Compute CCF
x = data['GDP_Growth'].values
y = data['Inflation'].values

# Normalize
x_norm = (x - np.mean(x)) / np.std(x)
y_norm = (y - np.mean(y)) / np.std(y)

# Cross-correlation
ccf_values = np.correlate(x_norm, y_norm, mode='full') / len(x)
lags = np.arange(-len(x)+1, len(x))

# Plot
fig, ax = plt.subplots(figsize=(12, 5))

# Only show lags -20 to 20
mask = (lags >= -20) & (lags <= 20)
ax.stem(lags[mask], ccf_values[mask], linefmt=COLORS['blue'], markerfmt='o', basefmt=' ')
ax.axhline(y=0, color='black', linestyle='-')
ax.axhline(y=1.96/np.sqrt(len(x)), color='red', linestyle='--', alpha=0.5)
ax.axhline(y=-1.96/np.sqrt(len(x)), color='red', linestyle='--', alpha=0.5)
ax.axvline(x=0, color='gray', linestyle=':', alpha=0.5)

ax.set_xlabel('Lag (k)')
ax.set_ylabel('Cross-Correlation')
ax.set_title('Cross-Correlation: GDP Growth and Inflation', fontweight='bold')

plt.tight_layout()
plt.show()

print("Interpretation:")
print("- Positive lag k: GDP leads Inflation by k periods")
print("- Negative lag k: Inflation leads GDP by k periods")
print("- Peak correlation shows dominant lead-lag relationship")

## Summary

### Key Takeaways

1. **VAR models** capture dynamic interdependencies between multiple time series
   - Each variable depends on its own lags AND lags of other variables
   - OLS estimation is efficient with identical regressors

2. **Stability** requires all eigenvalues inside the unit circle
   - Ensures stationarity and convergent impulse responses

3. **Granger causality** tests predictive content, not true causation
   - Useful for understanding lead-lag relationships
   - Sensitive to lag selection and omitted variables

4. **Impulse Response Functions** trace shock propagation
   - Requires identification (Cholesky, structural restrictions)
   - Shows dynamic multipliers over time

5. **FEVD** decomposes forecast variance by shock source
   - Shows relative importance of different shocks

6. **Cointegration** exists when I(1) variables share long-run equilibrium
   - Use VECM to model both short-run and long-run dynamics
   - Johansen test for testing cointegration rank

### Practical Workflow
1. Check stationarity (unit root tests)
2. Select lag order (information criteria)
3. Estimate VAR or VECM
4. Diagnostic checks (residual autocorrelation)
5. Granger causality tests
6. IRF and FEVD analysis
7. Forecasting